In [1]:
import random

import matplotlib.pyplot as plt
import numpy as np
from omegaconf import OmegaConf
import pandas as pd
import pywt
from skimage.transform import resize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# import wandb
import magnet

# import analysis

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(1, 64, num_layers=1, batch_first=True, bidirectional=False)
        self.fc_layers = nn.Sequential(
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :] # Get last output only (many-to-one)
        x = self.fc_layers(x)
        return x


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
YAML_CONFIG = OmegaConf.load("lstm.yaml")
CLI_CONFIG = OmegaConf.from_cli()
CONFIG = OmegaConf.merge(YAML_CONFIG, CLI_CONFIG)

In [4]:
# Reproducibility
random.seed(CONFIG.SEED)
np.random.seed(CONFIG.SEED)
torch.manual_seed(CONFIG.SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Setup CPU or GPU
if CONFIG.USE_GPU and not torch.cuda.is_available():
    raise ValueError("GPU not detected but CONFIG.USE_GPU is set to True.")
device = torch.device("cuda" if CONFIG.USE_GPU else "cpu")

In [5]:
# Setup dataset and dataloader
# NOTE(seungjaeryanlee): Load saved dataset for speed
dataset = magnet.PyTorchVoltageToCoreLossDataset(download_path="../../data/", download=True)
train_size = int(0.6 * len(dataset))
valid_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
kwargs = {'num_workers': 1, 'pin_memory': True} if CONFIG.USE_GPU else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=True, **kwargs)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=CONFIG.BATCH_SIZE, shuffle=False, **kwargs)

In [6]:
# Setup neural network and optimizer
net = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=CONFIG.LR)
# Log number of parameters
CONFIG.NUM_PARAMETERS = count_parameters(net)

In [7]:
# Setup wandb
# wandb.init(project="MagNet", config=OmegaConf.to_container(CONFIG), reinit=True)
# wandb.watch(net)

In [8]:
# Training
for epoch_i in range(1, CONFIG.NUM_EPOCH+1):
    # Train for one epoch
    epoch_train_loss = 0
    for inputs, labels in train_loader:
        inputs = inputs.unsqueeze(2) # LSTM requires 3 dimension: (batch size, # timesteps, # features)
        optimizer.zero_grad()
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()

    # Compute Validation Loss
    with torch.no_grad():
        epoch_valid_loss = 0
        for inputs, labels in valid_loader:
            inputs = inputs.unsqueeze(2) # LSTM requires 3 dimension: (batch size, # timesteps, # features)
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))

            epoch_valid_loss += loss.item()

    print(f"Epoch {epoch_i:2d} "
        f"Train {epoch_train_loss / len(train_dataset):.5f} "
        f"Valid {epoch_valid_loss / len(valid_dataset):.5f}")
#     wandb.log({
#         "train/loss": epoch_train_loss / len(train_dataset),
#         "valid/loss": epoch_valid_loss / len(valid_dataset),
#     })

Epoch  1 Train 0.00148 Valid 0.00130
Epoch  2 Train 0.00129 Valid 0.00114
Epoch  3 Train 0.00111 Valid 0.00100
Epoch  4 Train 0.00098 Valid 0.00087
Epoch  5 Train 0.00084 Valid 0.00076
Epoch  6 Train 0.00074 Valid 0.00067
Epoch  7 Train 0.00064 Valid 0.00058
Epoch  8 Train 0.00056 Valid 0.00051
Epoch  9 Train 0.00059 Valid 0.00044
Epoch 10 Train 0.00043 Valid 0.00038


In [9]:
# Evaluation
net.eval()
y_meas = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.unsqueeze(2) # LSTM requires 3 dimension: (batch size, # timesteps, # features)
        y_pred.append(net(inputs.to(device)))
        y_meas.append(labels.to(device))

y_meas = torch.cat(y_meas, dim=0)
y_pred = torch.cat(y_pred, dim=0)
test_mse_loss = F.mse_loss(y_meas, y_pred).item() / len(test_dataset)
print(f"Test Loss: {test_mse_loss:.8f}")
# wandb.log({"test/loss": test_mse_loss})

# Analysis
# wandb.log({
#     "test/prediction_vs_target": wandb.Image(analysis.get_scatter_plot(y_pred, y_meas)),
#     "test/prediction_vs_target_histogram": wandb.Image(analysis.get_two_histograms(y_pred, y_meas)),
#     "test/error_histogram": wandb.Image(analysis.get_error_histogram(y_pred, y_meas)),
# })

# Close wandb
# wandb.join()

Test Loss: 0.00004100
